In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.hudi:hudi-spark3.3-bundle_2.12:0.13.0 pyspark-shell'

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = (
    SparkSession.builder
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.hudi.catalog.HoodieCatalog",
    )
    .config("spark.sql.extensions", "org.apache.spark.sql.hudi.HoodieSparkSessionExtension")
    .getOrCreate()
)

23/04/05 10:36:54 WARN Utils: Your hostname, jordan resolves to a loopback address: 127.0.1.1; using 192.168.0.16 instead (on interface wlp0s20f3)
23/04/05 10:36:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/jordan/.cache/pypoetry/virtualenvs/spark-playground-X8qIQtop-py3.10/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jordan/.ivy2/cache
The jars for the packages stored in: /home/jordan/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c6254604-2468-421d-a636-dd60743a94c2;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.0 in central
downloading https://repo1.maven.org/maven2/org/apache/hudi/hudi-spark3.3-bundle_2.12/0.13.0/hudi-spark3.3-bundle_2.12-0.13.0.jar ...
	[SUCCESSFUL ] org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.0!hudi-spark3.3-bundle_2.12.jar (18841ms)
:: resolution report :: resolve 4130ms :: artifacts dl 18843ms
	:: modules in use:
	org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	----------------------

23/04/05 10:37:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/05 10:37:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/04/05 10:37:18 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [14]:
# Generate sample data
from pyspark.sql.functions import (
    rand,
    current_timestamp,
)

# Generate the DataFrame
df = (
    spark.range(1_000_000)
    .withColumn("timestamp", current_timestamp())
    .withColumn("random_float", rand(seed=42))
)

hudi_options = {
    "hoodie.table.name": "rand_nums",
    "hoodie.datasource.write.recordkey.field": "id",
    "hoodie.datasource.write.precombine.field": "timestamp",
    # "hoodie.datasource.write.operation": "upsert",
    # "write.insert.cluster": "true",
    # "hoodie.archive.merge.enable": "true",
}

(
    df.write.format("org.apache.hudi")
    .options(**hudi_options)
    .mode("overwrite")
    .save("/home/jordan/spark_playground/data/hudi/rand_nums")
)

23/04/05 10:43:41 WARN HoodieBackedTableMetadata: Metadata table was not found at path /home/jordan/spark_playground/data/hudi/rand_nums/.hoodie/metadata
23/04/05 10:43:41 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark


23/04/05 10:43:50 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_64_1 in memory.
23/04/05 10:43:50 WARN MemoryStore: Not enough space to cache rdd_64_1 in memory! (computed 0.0 B so far)
23/04/05 10:43:50 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_64_11 in memory.
23/04/05 10:43:50 WARN MemoryStore: Not enough space to cache rdd_64_11 in memory! (computed 0.0 B so far)
23/04/05 10:43:50 WARN BlockManager: Persisting block rdd_64_11 to disk instead.
23/04/05 10:43:50 WARN BlockManager: Persisting block rdd_64_1 to disk instead.
23/04/05 10:43:50 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_64_10 in memory.
23/04/05 10:43:50 WARN MemoryStore: Not enough space to cache rdd_64_10 in memory! (computed 0.0 B so far)
23/04/05 10:43:50 WARN BlockManager: Persisting block rdd_64_10 to disk instead.
23/04/05 10:43:50 WARN MemoryStore: F

23/04/05 10:44:08 WARN HoodieWriteConfig: Embedded timeline server is disabled, fallback to use direct marker type for spark
# WARNING: Unable to get Instrumentation. Dynamic Attach failed. You may add this JAR as -javaagent manually, or supply -Djdk.attach.allowAttachSelf
# WARNING: Unable to attach Serviceability Agent. Unable to attach even with module exceptions: [org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed., org.apache.hudi.org.openjdk.jol.vm.sa.SASupportException: Sense failed.]
23/04/05 10:44:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-hbase.properties,hadoop-metrics2.properties
